<a href="https://colab.research.google.com/github/Lakshay02020/Data_Collection/blob/main/WebScrapping_amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lakshay Singla

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

#Question 1

In [ ]:
def get_data(page_no):
  url = 'https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(page_no)+'?ie=UTF8&pg='+str(page_no)
  r = requests.get(url)
  soup = BeautifulSoup(r.content,'html.parser') #r.content byte code of the HTML .text 

  complete_page=[]

  for d in soup.findAll('div',attrs={'class':"zg-grid-general-faceout"}):
    book_name = d.find('div',attrs={'class':'_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y'})
    author_name = d.find('div',attrs={'class':'a-row a-size-small'})
    ratings = d.find('span',attrs={'class':'a-icon-alt'})
    link = d.find('a',attrs={'class':'a-link-normal'})
    sub = []
    
    #if a book name is not found it returns null
    if book_name is not None:
      sub.append(book_name.text)
    else:
      sub.append('NaN')
  
    #if a author name is not found it returns null
    if author_name is not None:
      sub.append(author_name.text)
    else:
      sub.append('NaN')
    
    #if a ratings is not found it returns null
    if ratings is not None:
      sub.append(ratings.text)
    else:
      sub.append('NaN')
    
    #if a link is not found it returns null
    if link is not None:
      sub.append('https://www.amazon.in/'+link.get('href'))
    else:
      sub.append('NaN')
    
    complete_page.append(sub)
  return complete_page

In [ ]:
def flatten(list1):
  list2=[]
  for sublist1 in list1:
    for sublist2 in sublist1:
      list2.append(sublist2)
  return list2

In [ ]:
def getReviews(soup):
  reviews = []
  for review in soup.findAll('div',attrs={'class':"a-expander-content reviewText review-text-content a-expander-partial-collapse-content"}):
      if(len(reviews)!=3):
        reviews.append(review.text.strip())
      else:
        break
  while (len(reviews)<3):
    reviews.append('')
  return reviews

In [ ]:
def getAllInfo(url):
  r = requests.get(url)
  if(r.status_code==200):
    soup = BeautifulSoup(r.content,'html.parser')
    temp = soup.find('div',attrs={'id':"detailBullets_feature_div"})
    info = {'Publisher':'','Language':'','Paperback':'','ISBN-10':'','ISBN-13':'','Item Weight':'','Dimensions':'','Customer Review 1':'','Customer Review 2':'','Customer Review 3':''}
    for list_item in temp.findAll('span',attrs={'class':'a-list-item'}):
      content = list_item.findAll('span')
      if len(content)==2:
        for i in list(info.keys()):
          if(i in content[0].text):
            info[i] = content[1].text
    reviews = getReviews(soup)
    for i in range(1,4):
      info['Customer Review '+str(i)] = reviews[i-1]
    info['Publisher'] = info['Publisher'].split(';')[0]
    return info
  else:
    print("Page render issue status code: ",r.status_code)
    return 'NaN'

In [ ]:
num_of_pages = 2
result=[]
for i in range(1,num_of_pages+1):
  result.append(get_data(1))

In [ ]:
df = pd.DataFrame(flatten(result),columns=['book name','author name','ratings','link to the book'])

In [ ]:
links = list(df['link to the book'])
allInfo=[]
for link in links:
  information = getAllInfo(link)
  if information != 'NaN':
    allInfo.append(information)

In [ ]:
if(len(allInfo)==0):
  print("There was an error while scraping either a captcha page popped up or too many requests were encountered by the amazon server giving an error 503")
dfISBN = pd.DataFrame(allInfo)
dfISBN

There was an error while scraping either a captcha page popped up or too many requests were encountered by the amazon server giving an error 503


""


#Question 2


In [ ]:
def scan(number):
    url="https://www.imdb.com/search/title/?release_date=2020-01-01,2020-12-31&sort=num_votes,desc&start="+str(number)+"&ref_=adv_prv"

    response = requests.get(url)
    soup = BeautifulSoup(response.content,'html.parser')

    complete_lst = []
    for d in soup.findAll('div',{'class':'lister-item-content'}):
      para_attri = d.findAll('p')

      Name = d.find('h3',{'class':'lister-item-header'})
      if Name is not None:
        Name = Name.text.strip().split("\n")[1]

      IMDB_Rating = d.find('div',{'name':'ir'})
      if IMDB_Rating is not None:
        IMDB_Rating = IMDB_Rating.text.strip()
      
      Description = para_attri[1]
      if Description is not None:
        Description = Description.text.strip()
      
      Duration = para_attri[0].find('span',{'class':'runtime'})
      if Duration is not None:
        Duration = Duration.text.strip().split(" ")[0]
      
      Genre = para_attri[0].find('span',{'class':'genre'})
      if Genre is not None:
        Genre = Genre.text.strip()
      
      Votes = para_attri[-1].find('span',{'name':'nv'})
      if Votes is not None:
        Votes = Votes.text.strip()

      MetaScore=d.find('span',{'class':'metascore'})
      if MetaScore is not None:
        MetaScore = MetaScore.text.strip()
      
      Directors = None
      if 'Director' in para_attri[2].text:
        Directors = (para_attri[2].text.split('|')[0].strip())[10:]
        Directors = Directors.replace('\n','')
      
      Stars = None
      if 'Stars' in para_attri[2].text:
        if 'Director' in para_attri[2].text:
          Stars = (para_attri[2].text.split('|')[1].strip())[7:]
          Stars = Stars.replace('\n','')
        else:
          Stars = (para_attri[2].text.strip())[7:]
          Stars = Stars.replace('\n','')
      
      info = {'Movie name':Name,'IMDB Rating':IMDB_Rating,'Description':Description,'Votes':Votes,'Director(s)':Directors,'Stars':Stars,'Metascore':MetaScore,'Movie Genre':Genre,'Total Duration (min)':Duration}
      complete_lst.append(info)
    return complete_lst

In [ ]:
final_1000 = []
number = 1
while len(final_1000)<1000:
  final_1000.extend(scan(number))
  number = number + 50

In [ ]:
IMDB_dataframe = pd.DataFrame(final_1000,columns = list(final_1000[0].keys()))
IMDB_dataframe

,Movie name,IMDB Rating,Description,Votes,Director(s),Stars,Metascore,Movie Genre,Total Duration (min)
0,Tenet,7.3,"Armed with only one word, Tenet, and fighting ...","503,254",Christopher Nolan,"John David Washington, Robert Pattinson, Eliza...",69,"Action, Sci-Fi, Thriller",150
1,The Queen's Gambit,8.6,"Orphaned at the tender age of nine, prodigious...","449,183",None,"Anya Taylor-Joy, Chloe Pirrie, Bill Camp, Marc...",None,Drama,60
2,Soul,8.0,"After landing the gig of a lifetime, a New Yor...","329,668","Pete Docter, Kemp Powers","Jamie Foxx, Tina Fey, Graham Norton, Rachel House",83,"Animation, Adventure, Comedy",100
3,Wonder Woman 1984,5.4,"Diana must contend with a work colleague, and ...","268,021",Patty Jenkins,"Gal Gadot, Chris Pine, Kristen Wiig, Pedro Pascal",60,"Action, Adventure, Fantasy",151
4,Birds of Prey,6.0,"After splitting with the Joker, Harley Quinn j...","240,076",Cathy Yan,"Margot Robbie, Rosie Perez, Mary Elizabeth Win...",60,"Action, Adventure, Comedy",109
...,...,...,...,...,...,...,...,...,...
995,BROS. On The Rocks,5.3,Matt and Tanner return for another night out i...,"4,035",Don Tjernagel,"Don Tjernagel, Matt Lee Ingebritson, Jordan Ab...",None,Comedy,95
996,Kalashnikov,6.6,"Wounded as a tank driver in 1941 during WWII, ...","4,026",Konstantin Buslov,"Yuriy Borisov, Olga Lerman, Artur Smolyaninov,...",None,"Biography, War",110
997,Ozark (2017–2022),8.2,"The FBI questions Ruth, tensions between Cosgr...","4,019",Alik Sakharov,"Jason Bateman, Laura Linney, Julia Garner, Sof...",None,"Crime, Drama, Thriller",61
998,Veneno,8.7,"A biographical story of La Veneno, transgender...","4,015",None,"Lola Rodríguez, Isabel Torres, Paca la Piraña,...",None,"Biography, Drama",421
